In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import os
import matplotlib.image as img
import tqdm
from vae import Basic_VAE
from encoder import Encoder
from decoder import Decoder

#### Create Dataset


In [2]:
cats = []
directory = "data/cats"
count = 0
for catpic in os.listdir(directory):
    if count < 1000:
        # read from image and convert to tensor
        im = torch.tensor(img.imread(os.path.join(directory, catpic))).float()
        # permute to (channels, height, width) for conv2d layer
        im = torch.permute(im, (2, 0, 1))
        # normalize to range between -1 and 1
        im = im / 128 - 1
        cats.append(im)
        count+=1
    else:
        break
cats = torch.stack(cats)
print(cats.shape)

torch.Size([1000, 3, 64, 64])


#### Test/Training/Validation Split


In [3]:
# Split the data into what we use for testing and not testing
traing_test_split = 0.75
training_test_cutoff = int(cats.shape[0] * traing_test_split + 1)
random_perm = torch.randperm(cats.shape[0])
not_test_tensor = cats[:training_test_cutoff]
testing_tensor = cats[training_test_cutoff:]
# Split the data into what we use for training and cross validation
training_cv_split = 0.8
training_cv_cutoff = int(not_test_tensor.shape[0] * training_cv_split)
training_tensor = not_test_tensor[:training_cv_cutoff]
cv_tensor = not_test_tensor[training_cv_cutoff:]
print(training_tensor.shape)
print(cv_tensor.shape)
print(testing_tensor.shape)

torch.Size([600, 3, 64, 64])
torch.Size([151, 3, 64, 64])
torch.Size([249, 3, 64, 64])


#### Choose Hyperparameters and Build Model


In [4]:
from vae import Basic_VAE
from encoder import Encoder
from decoder import Decoder
hidden_dims = [16, 32, 64, 128]
latent_dim = 64
in_dim = 3
model = Basic_VAE(in_dim, hidden_dims, latent_dim)
encoder = Encoder(in_dim, hidden_dims, latent_dim)
decoder = Decoder(latent_dim, hidden_dims)

[128, 64, 32, 16]
[128, 64, 32, 16]


In [6]:
# # Testing the model
model.forward(training_tensor)

# # Code to test out the encoder & decoder 
# mu, log_var = encoder.forward(training_tensor)
# print("mu: ", mu.shape)
# print("log_var: ", log_var.shape)

# reconstructed_img = decoder.forward(mu, log_var)


encoded.shape: torch.Size([600, 128, 3, 3])
encoded.shape after flatten: torch.Size([600, 1152])


[tensor([[[[ 4.7003e-02,  7.8419e-01, -7.4870e-01,  ...,  3.3646e-01,
            -1.1576e-01,  7.6757e-02],
           [-2.8521e-01,  8.2572e-01, -6.3444e-01,  ..., -1.2089e-01,
            -5.3247e-03, -1.9336e-01],
           [-2.0423e-01,  1.2930e-01, -5.9985e-01,  ...,  2.4378e-01,
             1.4875e-01,  7.0573e-02],
           ...,
           [ 6.4774e-01, -4.9139e-02, -5.8383e-01,  ...,  2.8547e-01,
            -3.8508e-01,  1.7492e-01],
           [-6.8796e-02, -1.0649e-01, -5.4175e-01,  ...,  1.4800e-01,
             9.4394e-02,  1.7160e-01],
           [ 2.5309e-01, -5.4308e-01, -7.7976e-01,  ...,  7.3276e-03,
            -2.9781e-01,  2.5925e-01]],
 
          [[ 1.7299e-01,  6.7214e-01,  8.2979e-03,  ..., -2.7474e-01,
             1.4346e-02, -3.3217e-02],
           [ 5.6829e-01,  8.7437e-01, -5.1566e-01,  ..., -6.3834e-01,
            -1.4784e-01, -3.5719e-01],
           [-3.6020e-01,  3.7714e-01,  7.1707e-01,  ...,  2.9984e-01,
            -3.3563e-02, -3.9339e-02],


#### Implement Loss Function

In [ ]:
def loss_function(self, reconstructed_img, input_img, mu, log_var, kld_weight=2):
    img_loss = F.mse_loss(reconstructed_img, input_img)
    # article on calculating kl divergence between 2 gaussians:
    # https://medium.com/@outerrencedl/variational-autoencoder-and-a-bit-kl-divergence-with-pytorch-ce04fd55d0d7
    kld_loss = torch.mean(
        torch.sum(-log_var + (log_var.exp() ** 2 + mu**2) / 2 - 1 / 2)
    )
    kld_loss *= kld_weight

    return img_loss + kld_loss

#### Implement Gradient Descent


In [ ]:
def gradient_descent(model, loss_func, x, y, xvalid, yvalid, lr=0.1, steps=5000):
    optimizer = optim.AdamW(model.parameters(), lr)

    losses = []
    valid_losses = []
    for _ in tqdm.trange(steps):
        model.train()
        loss = loss_func(model(x), y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        model.eval()
        valid_loss = loss_func(model(xvalid), yvalid)
        losses.append(loss.detach().numpy())
        valid_losses.append(valid_loss.detach().numpy())

    print(f"Final training loss: {losses[-1]}")

    return losses, valid_losses